### Full POP model works

In [19]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using Plots
using LinearAlgebra

Consider simplest possible Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

and dissipator is traceless:

$
A = \begin{pmatrix} a_1 + i b_1 &   a_2 + i b_2\\  a_3 + i b_3 & a_4 + i b_4
   \end{pmatrix} $

In [2]:
#using DynamicPolynomials

@polyvar e[1:2]
@polyvar h[1:2]

Hˢʸᵐᵇₐₙ = [ e[1]   0
            0      0. ]

Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     e[2]             ]

2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 e₁              h₁ + (0-1im)h₂
 h₁ + (0+1im)h₂  e₂

In [3]:
@polyvar a[1:4]
@polyvar b[1:4]

Aˢʸᵐᵇ = [ a[1] + im*b[1]           a[2] + im*b[2]
          a[3] + im*b[3]           a[4] + im*b[4]   ]

2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂ + (0+1im)b₂
 a₃ + (0+1im)b₃  a₄ + (0+1im)b₄

### Load exact data

In [4]:
γᵗˣᵗ = "0.079477"

parentdir = pwd()
data_dir = parentdir*"\\DATA\\"
println(data_dir)

ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵗˣᵗ)
ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵗˣᵗ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵗˣᵗ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵗˣᵗ)

@assert tᵍ == tᵉ == tˣ == tʸ 

t = convert(Vector{Float64}, tᵉ)
@assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]
Δt = t[2]-t[1]
t_steps = length(t)


C:\Users\Zakhar\Documents\GitHub\POP_fail\DATA\


1256

In [5]:
objₑₓ = 0
objₑₓₐₙ = 0

for ρ in [ρᵍ, ρᵉ, ρˣ, ρʸ]

    # Convert cut ρ series:
    ρ = convert(Vector{Matrix{ComplexF64}}, ρ)

    objₑₓ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
    
    objₑₓₐₙ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇₐₙ, [Aˢʸᵐᵇ])

end # of files (initial states) loop

objₑₓ 

0.19873096716801214a₁⁴ - 0.0017155126525017488a₁³a₂ + 0.0016619276546841692a₁³a₃ - 0.7949238686720486a₁³a₄ - 0.002020841077552012a₁³b₂ - 0.002699338341011222a₁³b₃ + 4.807681833454622a₁²a₂² - 12.58141127859036a₁²a₂a₃ + 0.005200122955322839a₁²a₂a₄ + 0.002020841077552012a₁²a₂b₁ - 0.0012575996083348737a₁²a₂b₂ - 0.0012575996083348737a₁²a₂b₃ + 0.0026993383410112183a₁²a₂b₄ + 10.980300647882432a₁²a₃² - 0.0050393679618701a₁²a₃a₄ + 0.002699338341011222a₁²a₃b₁ + 0.0012575996083348737a₁²a₃b₂ - 0.007419517759574447a₁²a₃b₄ + 1.1923858030080745a₁²a₄² + 0.005384025969196823a₁²a₄b₂ + 0.007419517759574447a₁²a₄b₃ + 0.3974619343360243a₁²b₁² - 0.0017155126525017488a₁²b₁b₂ + 0.0016619276546841692a₁²b₁b₃ - 0.7949238686720486a₁²b₁b₄ + 4.807765903889424a₁²b₂² + 12.581579419459953a₁²b₂b₃ - 0.0016619276546841694a₁²b₂b₄ + 10.980300647882432a₁²b₃² - 0.0050393679618701a₁²b₃b₄ + 1.1923858030080745a₁²b₄² - 0.005253707953140427a₁a₂³ + 0.005092952959687677a₁a₂²a₃ - 4.26462851964688a₁a₂²a₄ + 0.0012575996083348737a₁a₂²b₁

In [6]:
solₑₓⁿᵉʷ, best_methodₑₓⁿᵉʷ = LiPoSID.sos_min_newton(objₑₓ) 
Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓⁿᵉʷ)
Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓⁿᵉʷ)

************************TSSOS************************
TSSOS is launching...
optimum = 0.0005505172058373946

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Found a local optimal solution giving an upper bound: 0.011856943308885093 and a relative optimality gap: 0.011306426103047698.
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0005504181567844337
Global optimality certified!
No higher TSSOS hierarchy!


2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-1.19926+3.91461im)      (-0.246357+0.156774im)
 (-0.0577284+0.0316769im)  (-1.19931+3.91465im)

In [7]:
Hˢⁱᵈₑₓ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (18.1476+0.0im)        (0.482232-0.358003im)
 (0.482232+0.358003im)  (-6.97835-0.0im)

In [34]:
E, v = eigen(convert.(ComplexF64, Hˢⁱᵈₑₓ))
abs(E[1]-E[2])

25.154616081640242

In [27]:
function fix_hamiltonian_and_dissipator(H, L)
    H = convert.(ComplexF64, H)
    L = convert.(ComplexF64, L)
    a = - tr(L)/2
    Lᶠⁱˣᵉᵈ = L + I * a
    Hᶠⁱˣᵉᵈ = H + (a' * L - a * L')/2im
    return Hᶠⁱˣᵉᵈ, Lᶠⁱˣᵉᵈ
end

fix_hamiltonian_and_dissipator (generic function with 1 method)

In [28]:
Hᶠⁱˣ, Aᶠⁱˣ = fix_hamiltonian_and_dissipator(Hˢⁱᵈₑₓ, Aˢⁱᵈₑₓ)
Aᶠⁱˣ

2×2 Matrix{ComplexF64}:
 2.99496e-5-2.16036e-5im    -0.246357+0.156774im
 -0.0577284+0.0316769im   -2.99496e-5+2.16036e-5im

In [32]:
E, v = eigen(Hᶠⁱˣ)
abs(E[1]-E[2])

25.126102666258344

### Reguliarization do not help to find dissipator

In [39]:
α = 0.01
objₑₓʳᵉᵍ = objₑₓ + α * (e[1]^2 + e[2]^2 
                        + h[1]^2 + h[2]^2 
                        + a[1]^2 + b[1]^2 
                        + a[2]^2 + b[2]^2
                        + a[3]^2 + b[3]^2
                        + a[4]^2 + b[4]^2);

In [40]:
solₑₓʳᵉᵍ, best_methodₑₓʳᵉᵍ = LiPoSID.sos_min_newton(objₑₓʳᵉᵍ) 
Hˢⁱᵈₑₓʳᵉᵍ = subs(Hˢʸᵐᵇ, solₑₓʳᵉᵍ)
Aˢⁱᵈₑₓʳᵉᵍ = subs(Aˢʸᵐᵇ, solₑₓʳᵉᵍ)

************************TSSOS************************
TSSOS is launching...
optimum = 2.6824616387768496
Global optimality certified!
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 2.682460936630412
Global optimality certified!
No higher TSSOS hierarchy!


2×2 Matrix{Polynomial{true, ComplexF64}}:
 (4.30222-3.13134im)          (-0.000192667-0.000389076im)
 (0.000344464+0.000127132im)  (4.85622-2.17521im)

In [41]:
Hˢⁱᵈₑₓʳᵉᵍ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (9.57867+0.0im)              (-0.000234603+0.00013806im)
 (-0.000234603-0.00013806im)  (-9.57867-0.0im)

In [42]:
Eᶠⁱˣ, vᶠⁱˣ= eigen(convert.(ComplexF64, Hᶠⁱˣ))
abs(Eᶠⁱˣ[1]-Eᶠⁱˣ[2])

25.126102666258344

### Try to suppress diagonal of dissipator only

In [47]:
α = 0.1
objₑₓʳᵉᵍᵈ = objₑₓ + α * (a[1]^2 + b[1]^2 + a[4]^2 + b[4]^2);

In [48]:
solₑₓʳᵉᵍᵈ, best_methodₑₓʳᵉᵍ = LiPoSID.sos_min_newton(objₑₓʳᵉᵍᵈ) 
Hˢⁱᵈₑₓʳᵉᵍᵈ = subs(Hˢʸᵐᵇ, solₑₓʳᵉᵍᵈ)
Aˢⁱᵈₑₓʳᵉᵍᵈ = subs(Aˢʸᵐᵇ, solₑₓʳᵉᵍᵈ)

************************TSSOS************************
TSSOS is launching...
optimum = 0.0005505725589325444
Found a local optimal solution giving an upper bound: 0.01185694330882825 and a relative optimality gap: 0.011306370749895704.
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0005503811178959153
Found a local optimal solution giving an upper bound: 0.011856949901471125 and a relative optimality gap: 0.01130656878357521.
No higher TSSOS hierarchy!


2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-3.63272e-16-4.29738e-18im)  (-1.16272e-14-1.53471e-14im)
 (-6.06005e-16-9.20441e-16im)  (9.78033e-16+4.20506e-18im)

In [49]:
Hˢⁱᵈₑₓʳᵉᵍᵈ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (12.6098+0.0im)           (7.54484e-5-7.1159e-5im)
 (7.54484e-5+7.1159e-5im)  (-12.5163-0.0im)

In [50]:
Eʳᵉᵍᵈ, vʳᵉᵍᵈ = eigen(convert.(ComplexF64, Hˢⁱᵈₑₓʳᵉᵍᵈ))
abs(Eʳᵉᵍᵈ[1]-Eʳᵉᵍᵈ[2])

25.126072218584817